## Exercise 1

In this exercise we will understand the functioning of TF/IDF ranking. 

Implement the vector space retrieval model, based on the code framework provided below.

For testing we have provided a simple document collection with 5 documents in file bread.txt:

  DocID | Document Text
  ------|------------------
  1     | How to Bake Breads Without Baking Recipes
  2     | Smith Pies: Best Pies in London
  3     | Numerical Recipes: The Art of Scientific Computing
  4     | Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
  5     | Pastry: A Book of Best French Pastry Recipes

Now, for the query $Q = ``baking''$, find the top ranked documents according to the TF/IDF rank.

For further testing, use the collection __epfldocs.txt__, which contains recent tweets mentioning EPFL.

Compare the results also to the results obtained from the reference implementation using the scikit-learn library.

In [37]:
# Loading of libraries and documents

from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
nltk.download('punkt')

stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    
    # remove special characters !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    text = "".join([ch for ch in text if ch not in string.punctuation])
    
    # break up a sentence into a list of words and punctuation with no whitespaces
    # http://www.nltk.org/book/ch03.html
    # input is one sentence
    tokens = nltk.word_tokenize(text)
    
    # convert to lowercase words then have stemmers remove morphological affixes
    # why join them into a string if we are going to split them at the end ?
    # http://www.nltk.org/howto/stem.html
    return " ".join([stemmer.stem(word.lower()) for word in tokens])    
    
# Read a list of documents from a file. Each line in a file is a document
with open("bread.txt") as f:
    content = f.readlines()
    
# strip to remove whitespace - https://docs.python.org/3/library/stdtypes.html#str.strip
original_documents = [x.strip() for x in content] 

# after tokenize, split each document, resulting in a list of list
documents = [tokenize(d).split() for d in original_documents]

print(documents)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ha_ha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ha_ha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[['how', 'to', 'bake', 'bread', 'without', 'bake', 'recip'], ['smith', 'pie', 'best', 'pie', 'in', 'london'], ['numer', 'recip', 'the', 'art', 'of', 'scientif', 'comput'], ['bread', 'pastri', 'pie', 'and', 'cake', 'quantiti', 'bake', 'recip'], ['pastri', 'a', 'book', 'of', 'best', 'french', 'pastri', 'recip']]


In [34]:
# https://stackoverflow.com/questions/9197844/maintain-count-in-python-list-comprehension
# numbers = [[1,2,3,4,5,6],[2,3,4,5,6]]
# [[x, sublist.count(x)] for sublist in numbers for x in sublist]

In [66]:
#[item for sublist in documents for item in sublist]
#[item for sublist in documents if 'how' in sublist]
# https://stackoverflow.com/questions/17657720/python-list-comprehension-double-for

# collection = [['a','b','b','c','d','e','f'],['b','b','d','f']]
# [item for sublist in collection for item in sublist]
# [item for sublist in collection if 'a' in sublist]
# [item for sublist in collection for item in sublist if 'b' == item]
print(len(vocabulary))

17


In [67]:
# TF/IDF code

# create the vocabulary, an unordered collection of unique elements
vocabulary = set([item for sublist in documents for item in sublist])

# remove all stopwords 
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]

# sort to alphabetical order
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {} # initialize dictionary
    num_documents = len(documents)    
    for i, term in enumerate(vocabulary):                
        # compute the number of documents that contain each term
        idf[term] = math.log(num_documents/sum([term in x for x in documents]), math.e)    
                
    return idf

# Function to generate the vector for a document (with normalisation)
# compute tf-idf
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i, term in enumerate(vocabulary):
        vector[i] = document.count(term)/max_count * idf[term]
    return vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i];
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy / (math.sqrt(sumxx)*math.sqrt(sumyy))
    return result

# computing the search result (get the topk documents)
def search_vec(query, topk):
    
    # process text
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    
    # compute cosine similarity  
    # scores is a list the score for each document formatted as [score, document_index]
    # scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores = []
    for i,d in enumerate(document_vectors):
        scores.append([cosine_similarity(query_vector, d), i])
    
    # sort scores by its first element
    scores.sort(key=lambda x: -x[0])
    for i in range(topk):
            print(original_documents[scores[i][1]], scores[i][0])

# HINTS
# natural logarithm function
#     math.log(n,math.e)
# Function to count term frequencies in a document
#     Counter(document)
# most common elements for a list
#     counts.most_common(1)
print(document_vectors)
print(search_vec('baking recipe',5))

[[0.0, 1.8325814637483102, 0.0, 0.0, 0.9162907318741551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22314355131420976, 0.0, 0.0, 1.6094379124341003], [0.0, 0.0, 0.9162907318741551, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6094379124341003, 0.0, 0.0, 1.8325814637483102, 0.0, 0.0, 0.0, 1.6094379124341003, 0.0], [1.6094379124341003, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6094379124341003, 0.0, 0.0, 1.6094379124341003, 0.0, 0.0, 0.0, 0.22314355131420976, 1.6094379124341003, 0.0, 0.0], [0.0, 0.9162907318741551, 0.0, 0.0, 0.9162907318741551, 1.6094379124341003, 0.0, 0.0, 0.0, 0.0, 0.9162907318741551, 0.9162907318741551, 1.6094379124341003, 0.22314355131420976, 0.0, 0.0, 0.0], [0.0, 0.0, 0.9162907318741551, 1.6094379124341003, 0.0, 0.0, 0.0, 1.6094379124341003, 0.0, 0.0, 1.8325814637483102, 0.0, 0.0, 0.22314355131420976, 0.0, 0.0, 0.0]]
How to Bake Breads Without Baking Recipes 0.7010969225741529
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes 0.32179527698963223
Pastry: A Book of Best French Pastry Recipes 

In [100]:
# Reference code using scikit-learn

# Initialize settings
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english')

# Compute tf-idf 
features = tf.fit_transform(original_documents)

# Convert features to "matrix form"
npm_tfidf = features.todense()
#print(tf.get_feature_names()) #print feature names
#print(features)               #print tf-idf scores

# Transform query
new_features = tf.transform(['numerical'])

# Compute cosine similarity
cosine_similarities = linear_kernel(new_features, features).flatten()

# Sort and retain indices
related_docs_indices = cosine_similarities.argsort()[::-1]

# Print top scores
topk = 5
for i in range(topk):
    print(original_documents[related_docs_indices[i]], cosine_similarities[related_docs_indices[i]])

Numerical Recipes: The Art of Scientific Computing 0.481270078046231
Pastry: A Book of Best French Pastry Recipes 0.0
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes 0.0
Smith Pies: Best Pies in London 0.0
How to Bake Breads Without Baking Recipes 0.0



## Exercise 2

Implement probabilistic retrieval model based on the query likelihood language model, using a mixture model between the documents and the collection, both weighted at 0.5. Maximum likelihood estimation (mle) is used to estimate both as unigram models. You can use the code framework provided below.

Now, for the query $Q = ``baking''$, find the top ranked documents according to the probabilistic rank.

Compare the results with TF/IDF ranking.

In [113]:
# Probabilistic retrieval code

# smoothing parameter
lmbda = 0.5

# term frequency
def tf(word, document):
    return document.count(word)/len(document)

# collection size
# total number of terms in collection
def collection_size(documents):
    cs = 0
    for d in documents:
        cs += len(documents)
    return cs

# collection frequency
def cf(word, documents):
    cf = 0
    for d in documents:
        cf += d.count(word)
    return cf/collection_size(documents)

# probabilistic relevance
# probability that the document generated the query
def query_prob(query, document):
    prob = 1
    
    for q in query:
        prob *= (lmbda * tf(q, document) + (1-lmbda) * cf(q, documents))
    
    return prob

# computing the search result
def search_prob(query, k):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    scores = [[query_prob(q, documents[d]),d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    for i in range(len(k)):
            print(original_documents[scores[i][1]], scores[i][0])

# HINTS
# counting occurrences of a word in a document:
#     document.count(word)
# length of a document:
#     len(document)
# querying:
#     print(search_prob("How", documents))
print(search_prob("Numerical Breads Baking", documents))

How to Bake Breads Without Baking Recipes 0.00045208163265306124
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes 0.000251125
Numerical Recipes: The Art of Scientific Computing 0.00021942857142857143
Smith Pies: Best Pies in London 4.8e-05
Pastry: A Book of Best French Pastry Recipes 4.8e-05
None


## Exercise 3
Following the notation used in class, let us denote the set of terms by $T=\{k_i|i=1,...,m\}$, the set of documents by $D=\{d_j |j=1,...,n\}$, and let $d_i=(w_{1j},w_{2j},...,w_{mj})$. We are also given a query  $q=(w_{1q},w_{2q},...,w_{mq})$. In the lecture we studied that, 

$sim(q,d_j) = \sum^m_{i=1} \frac{w_{ij}}{|d_j|}\frac{w_{iq}}{|q|}$ .  (1)

Another way of looking at the information retrieval problem is using a probabilistic approach. The probabilistic view of information retrieval consists of determining the conditional probability $P(q|d_j)$ that for a given document $d_j$ the query by the user is $q$. So, practically in probabilistic retrieval when a query $q$ is given, for each document it is evaluated how probable it is that the query is indeed relevant for the document, which results in a ranking of the documents.

In order to relate vector space retrieval to a probabilistic view of information retrieval, we interpret the weights in Equation (1) as follows:

-  $w_{ij}/|d_j|$ can be interpreted as the conditional probability $P(k_i|d_j)$ that for a given document $d_j$ the term $k_i$ is important (to characterize the document $d_j$).

-  $w_{iq}/|q|$ can be interpreted as the conditional probability $P(q|k_i)$ that for a given term $k_i$ the query posed by the user is $q$. Intuitively, $P(q|k_i)$ gives the amount of importance given to a particular term while querying. <font color="red">Why not $P(k_i|q)$</font>

With this interpretation you can rewrite Equation (1) as follows:

$sim(q,d_j) = \sum^m_{i=1} P(k_i|d_j)P(q|k_i)$ (2)

### Question a
Show that indeed with the probabilistic interpretation of weights of vector space retrieval, as given in Equation (2), the similarity computation in vector space retrieval results exactly in the probabilistic interpretation of information retrieval, i.e., $sim(q,d_j)= P(q|d_j)$.
Given that $d_j$ and $q$ are conditionally independent, i.e., $P(d_j \cap q|ki) = P(d_j|k_i)P(q|k_i)$. You can assume existence of joint probability density functions wherever required. (Hint: You might need to use Bayes theorem)

### Solution

\begin{align}
P(q|d_j) &= \dfrac{P(d_j|q)P(q)}{P(d_j)} \\
\sum^m_{i=1} P(q,k_i|d_j) &= \dfrac{\sum^m_{i=1} P(q,k_i,d_j)}{P(d_j)} \\
&= \dfrac{\sum^m_{i=1} P(d_j,q|k_i)P(k_i)}{P(d_j)} \\
&= \dfrac{\sum^m_{i=1} P(d_j|k_i)P(q|k_i)P(k_i)}{P(d_j)}
\end{align}

\begin{align}
sim(q,d_j) &= \sum^m_{i=1} P(k_i|d_j)P(q|k_i) \\
&= \dfrac{\sum^m_{i=1} P(d_j|k_i)P(k_i)P(q|k_i) }{P(d_j)}
\end{align}

### Question b
Using the expression derived for $P(q|d_j)$ in (a), obtain a ranking (documents sorted in descending order of their scores) for the documents $P(k_i|d_1) = (0, 1/3, 2/3)$, $P(k_i|d_2) =(1/3, 2/3, 0)$, $P(k_i|d_3) = (1/2, 0, 1/2)$, and $P (k_i|d_4) = (3/4, 1/4, 0)$ and the query $P(q|k_i) = (1/5, 0, 2/3)$.

\begin{align}
sim(q,d_1) &= \sum^m_{i=1} P(k_i|d_j)P(q|k_i) \\
&= 0*\dfrac{1}{5} + \dfrac{1}{3}*0 + \dfrac{2}{3}*\dfrac{2}{3} \\
&= \dfrac{4}{9}
\end{align}

\begin{align}
sim(q,d_2) &= \sum^m_{i=1} P(k_i|d_j)P(q|k_i) \\
&= \dfrac{1}{3}*\dfrac{1}{5} + \dfrac{2}{3}*0 + 0*\dfrac{2}{3} \\
&= \dfrac{1}{15}
\end{align}

\begin{align}
sim(q,d_3) &= \sum^m_{i=1} P(k_i|d_j)P(q|k_i) \\
&= \dfrac{1}{2}*\dfrac{1}{5} + 0*0 + \dfrac{1}{2}*\dfrac{2}{3} \\
&= \dfrac{13}{30}
\end{align}